Importing necessary libraries

In [15]:
from fastapi import FastAPI
from pydantic import BaseModel, ValidationError
from pathlib import Path
import xgboost as xg
import pickle
import numpy as np
import json

Preprocessing data


In [16]:
X_json = {
    "data": {
        "property_type": "house",
        "floor": 3,
        "bedrooms_number": 2,
        "habitable_surface": 1100,
        "bathroom_number": 1,
        "condition": 3
    }
}

def processing(X):
    X['data']['property_type'] = X['data']['property_type'].lower()
    if not isinstance(X['data']['floor'], int):
        print('Check your data. Expected "floor" to be an integer.')
        return None
    
    if X['data']['property_type'] == 'apartment':
        pass
    elif X['data']['property_type'] == 'house':
        del X['data']['floor']
    else:
        print('Check your data. Expected property type House or Apartment.')
        return None
    
    if not isinstance(X['data']['bedrooms_number'], int):
        print('Check your data. Expected "bedrooms_number" to be an integer.')
        return None
    
    if not isinstance(X['data']['habitable_surface'], (int, float)):
        print('Check your data. Expected "habitable_surface" to be an integer or a float.')
        return None
    
    if not isinstance(X['data']['bathroom_number'], int):
        print('Check your data. Expected "bathroom_number" to be an integer.')
        return None
    
    if not 0 <= X['data']['condition'] <= 5:
        print('Check your data. Expected "condition" to be an integer from 0 to 5.')
        return None
    
    X_values = list(X['data'].values())
    X_array = np.array(X_values)
    return X_array


X_pred_pr = processing(X_json)

print(X_pred_pr)

['house' '2' '1100' '1' '3']


Prediction: open regressor file

In [17]:
def open_reg():
    if X_pred_pr[0] == 'apartment':
        path_open_pickle_apart = Path.cwd().parent.parent / "models" / "xgbr_apart.pickle"
        with open(path_open_pickle_apart, 'rb') as file:
            reg_apart = pickle.load(file)
        return reg_apart
    else:
        path_open_pickle_houses = Path.cwd().parent.parent / "models" / \
            "xgbr_houses.pickle"
        with open(path_open_pickle_houses, 'rb') as file:
            reg_houses = pickle.load(file)
        return reg_houses


xgbr = open_reg()
display(xgbr)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=10, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

prediction: making a predition, converting into json file

In [18]:
def predict_new(X, reg):
    X = np.array(X)[1:]
    X = X.astype(np.float64)
    X = X.reshape(1, -1)
    y_pred_array = reg.predict(X)
    y_pred_dic = {
        "prediction": round(float(y_pred_array[0]), 2),
        "status_code": 200,
    }
    y_pred_json = json.dumps(y_pred_dic)
    return y_pred_json


y_pred = predict_new(X_pred_pr, xgbr)

display(y_pred)

'{"prediction": 142827.42, "status_code": 200}'

Creating app and class

In [19]:
app = FastAPI()

class PropertyData(BaseModel):
    property_type: str
    floor: int
    bedrooms_number: int
    habitable_surface: float
    bathroom_number: int
    condition: int

class InputData(BaseModel):
    data: PropertyData

Making the get request

In [20]:
@app.get('/')
def whatever_func():
    text = '''This is an API for price prediction for Belgian real estate.\n
This model is built on base of trained data of more that 15000 Belgian real estate properties both apartments and 
houses using XGBoos regressor.\nTo make a prediction of a property price you need to input data in JSON format as follows:\n
{"data":
    {"property_type": string. Expected "House" or "Apartment",
    "floor": integer,
    "bedrooms_number": integer,
    "habitable_surface": float,
    "bathroom_number":integer,
    "condition": integer. Expected from 0 to 5. O - very bad condition, to be done up, 5 - new.
        }
            } '''
    return text

x = whatever_func()
print(x)

This is an API for price prediction for Belgian real estate.

This model is built on base of trained data of more that 15000 Belgian real estate properties both apartments and 
houses using XGBoos regressor.
To make a prediction of a property price you need to input data in JSON format as follows:

{"data":
    {"property_type": string. Expected "House" or "Apartment",
    "floor": integer,
    "bedrooms_number": integer,
    "habitable_surface": float,
    "bathroom_number":integer,
    "condition": integer. Expected from 0 to 5. O - very bad condition, to be done up, 5 - new.
        }
            } 


Making the post request

In [21]:
@app.post('/calc')
async def predict_price(data: InputData):
    X_pred_pr = processing(data)
    xgbr = open_reg(X_pred_pr)
    y_pred = predict_new(X_pred_pr, xgbr)
    return y_pred

print(y_pred)

{"prediction": 142827.42, "status_code": 200}
